Bulk processing Sentinel products to PNG images

In [19]:
import geopandas as gpd
import folium
import os
from PIL import Image
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
import matplotlib.pyplot as plt
import numpy as np
import os
#from geojson import Point
from geojson import Polygon
import geojson
import tempfile
import datetime as dt
from shapely.geometry import MultiPolygon, Polygon
import rasterio as rio
from rasterio import plot
import rasterio.mask
import fiona
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from skimage import exposure
from pathlib import Path

In [20]:
# Create a path to the exported folder
path = Path('./data')
k=0
# traverse root directory, and list directories as dirs and files as files
for root, dirs, files in os.walk("./data"):
    path = root.split(os.sep)
    print((len(path) - 1) * '---', os.path.basename(root))

    if os.path.basename(root) == 'IMG_DATA':
        k+=1
        for file in files:
            print(len(path) * '---', file)

            if 'B02' in file: band2 = rasterio.open(os.path.join(root,file), driver='JP2OpenJPEG') #blue
            if 'B03' in file: band3 = rasterio.open(os.path.join(root,file), driver='JP2OpenJPEG') #green
            if 'B04' in file: band4 = rasterio.open(os.path.join(root,file), driver='JP2OpenJPEG') #red
            if 'B08' in file: band8 = rasterio.open(os.path.join(root,file), driver='JP2OpenJPEG') #nir
            if 'B12' in file: band12 = rasterio.open(os.path.join(root,file), driver='JP2OpenJPEG') #swir


        if band4 is not None:
            #number of raster bands
            print(band4.count)
            #number of raster columns
            print(band4.width)
            #number of raster rows
            print(band4.height)

            #type of raster byte
            print(band4.dtypes[0])
            #raster sytem of reference
            print(band4.crs)
            #raster transform parameters
            print(band4.transform)

            #export true color image
            trueColor = rasterio.open('./Output/Cache/SentinelTemp.tiff','w',driver='Gtiff',
                                     width=band4.width, height=band4.height,
                                     count=3,
                                     crs=band4.crs,
                                     transform=band2.transform,
                                     dtype=band2.dtypes[0]
                                     )
            trueColor.write(band2.read(1),1) #blue
            trueColor.write(band3.read(1),2) #green
            trueColor.write(band4.read(1),3) #red
            trueColor.close()

            # Mask area of study

            # Load polygon vector of Gozo to clip satellite image
            boundsdata = r'./data/delimara_close_area_geojson.geojson'
            region = gpd.read_file(boundsdata)
            region

            nReserve_proj = region.to_crs({'init': 'epsg:32633'})


            with rio.open("./Output/Cache/SentinelTemp.tiff") as src:
                out_image, out_transform = rio.mask.mask(src, nReserve_proj.geometry,crop=True)
                out_meta = src.meta.copy()
                out_meta.update({"driver": "GTiff",
                             "height": out_image.shape[1],
                             "width": out_image.shape[2],
                             "transform": out_transform,
                             "dtype": "uint16"})

            # Rescale the image (divide by 10000 to convert to [0:1] reflectance
            image = np.array(out_image[[2,1], :, :])
            p2, p98 = np.percentile(image, (2,98))
            image = exposure.rescale_intensity(image, in_range=(p2, p98)) / 100000

            rgb = np.array(image.transpose(1,2))

            #Plot and save image with Matplotlib
            fig = plt.figure(figsize=(20,16))
            ax=fig.add_subplot(1,1,1)
            plt.axis('off')
            plt.imshow(rgb)

            #Save scaled reflectance image
            # Call 'savefig' to save this plot to an image file
            file_to_save = 'Rendered_' + str(k).zfill(3) +'.png'
            plt.savefig(os.path.join('./Output/PROCESSED', file_to_save), bbox_inches='tight', transparent=True, pad_inches=0)
            plt.show()

 data
--- S2A_MSIL1C_20210315T094031_N0209_R036_T33SVV_20210315T110237.SAFE
------ AUX_DATA
------ DATASTRIP
--------- DS_VGS4_20210315T110237_S20210315T094345
------------ QI_DATA
------ GRANULE
--------- L1C_T33SVV_A029920_20210315T094345
------------ AUX_DATA
------------ IMG_DATA
--------------- T33SVV_20210315T094031_B01.jp2
--------------- T33SVV_20210315T094031_B02.jp2
--------------- T33SVV_20210315T094031_B03.jp2
--------------- T33SVV_20210315T094031_B04.jp2
--------------- T33SVV_20210315T094031_B05.jp2
--------------- T33SVV_20210315T094031_B06.jp2
--------------- T33SVV_20210315T094031_B07.jp2
--------------- T33SVV_20210315T094031_B08.jp2
--------------- T33SVV_20210315T094031_B09.jp2
--------------- T33SVV_20210315T094031_B10.jp2
--------------- T33SVV_20210315T094031_B11.jp2
--------------- T33SVV_20210315T094031_B12.jp2
--------------- T33SVV_20210315T094031_B8A.jp2
--------------- T33SVV_20210315T094031_TCI.jp2
1
10980
10980
uint16
EPSG:32633
| 10.00, 0.00, 399960.00|

ValueError: axes don't match array

Read images from a directory and construct video

In [ ]:
# Creaating Video from Images using OpenCV-Python
import cv2
import numpy as np
import glob

img_array = []
for filename in glob.glob('./Output/Processed/*.png'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)


out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'MJPG'), 1, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()


In [ ]:
# Algorithm for blending multiple images using OpenCV

import glob
import numpy as np
import cv2

# Import all image files with the .jpg extension
files = glob.glob ('./Output/Processed/*.png')
image_data = []
for my_file in files:
    this_image = cv2.imread(my_file, 1)
    image_data.append(this_image)

# Calculate blended image
dst = image_data[0]
for i in range(len(image_data)):
    if i == 0:
        pass
    else:
        alpha = 1.0/(i + 1)
        beta = 1.0 - alpha
        dst = cv2.addWeighted(image_data[i], alpha, dst, beta, 0.0)

# Save blended image
cv2.imwrite('./Output/Processed/blended.png', dst)
